# Extension - RRT: basic finding example
* Path finding is using sample-based (RRT) methods rather than constrained optimization to find a collision free path
* Path finding is much easier if a final robot pose is given. We here use IK (formulated as KOMO problem) to first compute a final configuration 'qT'. (Path optimization does this jointly with optimizing the star_path.)
* Then we can pass the current state and qT to a bi-directional RRT to find a collision free path.
* Note that BotOp (the interface to sim/real) is opened only after we computed the motion. We simply pass the motion to be played by the sim/real.

In [2]:
import robotic as ry
import time
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 3}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.helloworld()

Hello, World!


In [ ]:
# Create a new configuration
C = ry.Config()

C.addFile("/home/monke/Xrai/maze.g")

# Add the moving "ego" frame with constraints on its motion
C.addFrame("ego", "floorwalls") \
    .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
    .setRelativePosition([0.2, 0.2, 0.4]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("goal1") \
    .setPosition(C.getFrame('ego').getPosition()+[2, -3, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

# C.view()
q0 = [0.2, 0.2, 0.4]
qT = C.getFrame('goal1').getPosition()

ry.params_clear()
ry.params_add({'rrt/stepsize':0.05, 'rrt/verbose': 0,'rrt/maxIters':15000}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.setProblem(C, [q0], [qT])
ret = rrt.solve()
ret_star = rrt.psbi_solve()
print(ret)
path = ret.x
star_path = ret_star.x
print(ret_star)




In [ ]:
import math
# Create a new configuration
C = ry.Config()

C.addFile("/home/monke/Xrai/maze.g")

# Add the moving "ego" frame with constraints on its motion
C.addFrame("ego", "floorwalls") \
    .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
    .setRelativePosition([0.2, 0.2, 0.4]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("goal1") \
    .setPosition(C.getFrame('ego').getPosition()+[1, -3, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

C.addFrame("goal2") \
    .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

C.view()
# Visualize the solution path
if ret.feasible:
    # print("path", path)
    for i in range(len(path)):
        # Add a green sphere at each path point
        C.addFrame(f"path_{i}").setPosition(path[i]).setShape(ry.ST.sphere, [0.02]).setColor([0, 1, 0])
        if i > 0:
            # Compute the direction vector from the previous point to the current point
            start_point = path[i-1]
            end_point = path[i]
            direction = [end_point[0] - start_point[0], end_point[1] - start_point[1], end_point[2] - start_point[2]]
            length = sum([d**2 for d in direction])**0.5  # Euclidean distance between points
            
            # Normalize the direction vector
            direction = [d / length for d in direction]

            # Calculate the position for the cylinder (midpoint between the points)
            midpoint = [(start_point[j] + end_point[j]) / 2 for j in range(3)]
            
            # Find the rotation axis (from the current orientation to the direction vector)
            # Assume the initial orientation of the cylinder is aligned along the Z-axis
            initial_direction = [0, 0, 1]
            dot_product = sum([initial_direction[j] * direction[j] for j in range(3)])
            cross_product = [
                initial_direction[1] * direction[2] - initial_direction[2] * direction[1],
                initial_direction[2] * direction[0] - initial_direction[0] * direction[2],
                initial_direction[0] * direction[1] - initial_direction[1] * direction[0]
            ]
            cross_length = sum([cp**2 for cp in cross_product])**0.5
            angle = math.acos(dot_product)

            # Normalize the cross product vector (axis of rotation)
            if cross_length > 0:
                axis = [cp / cross_length for cp in cross_product]
            else:
                axis = [0, 0, 1]  # Default axis in case the direction vector is already aligned

            # Convert the axis-angle representation to a quaternion
            half_angle = angle / 2.0
            qw = math.cos(half_angle)
            qx, qy, qz = [axis[j] * math.sin(half_angle) for j in range(3)]

            # Create the quaternion
            quaternion = [qw, qx, qy, qz]

            # Add the cylinder with the quaternion rotation
            C.addFrame(f"path_line_{i-1}_{i}") \
                .setPosition(midpoint) \
                .setShape(ry.ST.cylinder, [length,0.01]) \
                .setColor([1, 1, 0]) \
                .setQuaternion(quaternion)
        if i % 5 == 0:
            C.view()

if ret_star.feasible:
    # print("star_path", star_path)
    for i in range(len(star_path)):
        # Add a green sphere at each star_path point
        if i == len(star_path) -1:
                print("color updated")
                print("position : ",star_path[i] )
                C.addFrame(f"star_path_{i}_last").setPosition(star_path[i]).setShape(ry.ST.sphere, [1.12]).setColor([0, 0 , 0])
        else:
             C.addFrame(f"star_path_{i}").setPosition(star_path[i]).setShape(ry.ST.sphere, [0.02]).setColor([0, 0 , 1])


        if i > 0:
            # Compute the direction vector from the previous point to the current point
            start_point = star_path[i-1]
            end_point = star_path[i]
            direction = [end_point[0] - start_point[0], end_point[1] - start_point[1], end_point[2] - start_point[2]]
            length = sum([d**2 for d in direction])**0.5  # Euclidean distance between points
            
            # Normalize the direction vector
            direction = [d / length for d in direction]

            # Calculate the position for the cylinder (midpoint between the points)
            midpoint = [(start_point[j] + end_point[j]) / 2 for j in range(3)]
            
            # Find the rotation axis (from the current orientation to the direction vector)
            # Assume the initial orientation of the cylinder is aligned along the Z-axis
            initial_direction = [0, 0, 1]
            dot_product = sum([initial_direction[j] * direction[j] for j in range(3)])
            cross_product = [
                initial_direction[1] * direction[2] - initial_direction[2] * direction[1],
                initial_direction[2] * direction[0] - initial_direction[0] * direction[2],
                initial_direction[0] * direction[1] - initial_direction[1] * direction[0]
            ]
            cross_length = sum([cp**2 for cp in cross_product])**0.5
            angle = math.acos(dot_product)

            # Normalize the cross product vector (axis of rotation)
            if cross_length > 0:
                axis = [cp / cross_length for cp in cross_product]
            else:
                axis = [0, 0, 1]  # Default axis in case the direction vector is already aligned

            # Convert the axis-angle representation to a quaternion
            half_angle = angle / 2.0
            qw = math.cos(half_angle)
            qx, qy, qz = [axis[j] * math.sin(half_angle) for j in range(3)]

            # Create the quaternion
            quaternion = [qw, qx, qy, qz]
            color = [1,0,1]
           
            # Add the cylinder with the quaternion rotation
            C.addFrame(f"star_path_line_{i-1}_{i}") \
                .setPosition(midpoint) \
                .setShape(ry.ST.cylinder, [length,0.01]) \
                .setColor([1, 0, 1]) \
                .setQuaternion(quaternion)
        if i % 5 == 0:
            C.view()

In [ ]:
import numpy as np
x_list = [0.9,1.0,1.1,1.2,1.3,1.4,1.5]
for x_dist in x_list:
    # Metrics storage
    total_path_length_rrt = 0
    total_evals_rrt = 0
    total_path_length_psbi = 0
    total_evals_psbi = 0
    total_path_length_star = 0
    total_evals_star = 0
    total_path_length_single_star = 0
    total_evals_single_star = 0

    successful_paths_rrt = 0
    successful_paths_psbi = 0
    successful_paths_star = 0
    successful_paths_single_star = 0

    # Run tests for 100 iterations
    for i in range(100):
            # Configuration setup
        C = ry.Config()
        C.addFile("/home/monke/Xrai/maze.g")

        # Add the moving "ego" frame with constraints on its motion
        C.addFrame("ego", "floorwalls") \
            .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
            .setRelativePosition([0.2, 0.2, 0.4]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([0, 1., 1.]) \
            .setContact(1)

        C.addFrame("goal1") \
            .setPosition(C.getFrame('ego').getPosition() + [x_dist, -3, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        C.addFrame("goal2") \
            .setPosition(C.getFrame('goal1').getPosition() + [4, 0, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        q0 = [0.2, 0.2, 0.4]
        qT = C.getFrame('goal1').getPosition()

        # Parameters setup
        ry.params_clear()
        ry.params_add({'rrt/stepsize': 0.05, 'rrt/verbose': 0, 'rrt/maxIters': 15000})

        # PathFinder setup
        rrt = ry.PathFinder()
        rrt.setProblem(C, [q0], [qT])
        # Solve using rrt.solve
        ret = rrt.solve()
        if ret.feasible:
            path = ret.x
            total_path_length_rrt += np.sum([np.linalg.norm(np.array(path[j+1]) - np.array(path[j])) for j in range(len(path)-1)])
            total_evals_rrt += ret.evals
            successful_paths_rrt += 1

            # Configuration setup
        C = ry.Config()
        C.addFile("/home/monke/Xrai/maze.g")

        # Add the moving "ego" frame with constraints on its motion
        C.addFrame("ego", "floorwalls") \
            .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
            .setRelativePosition([0.2, 0.2, 0.4]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([0, 1., 1.]) \
            .setContact(1)

        C.addFrame("goal1") \
            .setPosition(C.getFrame('ego').getPosition() + [x_dist, -3, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        C.addFrame("goal2") \
            .setPosition(C.getFrame('goal1').getPosition() + [4, 0, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        q0 = [0.2, 0.2, 0.4]
        qT = C.getFrame('goal1').getPosition()

        # Parameters setup
        ry.params_clear()
        ry.params_add({'rrt/stepsize': 0.05, 'rrt/verbose': 0, 'rrt/maxIters': 15000})

        # PathFinder setup
        rrt = ry.PathFinder()
        rrt.setProblem(C, [q0], [qT])
        # Solve using rrt.psbi_solve
        ret_star = rrt.psbi_solve()
        if ret_star.feasible:
            star_path = ret_star.x
            total_path_length_psbi += np.sum([np.linalg.norm(np.array(star_path[j+1]) - np.array(star_path[j])) for j in range(len(star_path)-1)])
            total_evals_psbi += ret_star.evals
            successful_paths_psbi += 1

            # Configuration setup
        C = ry.Config()
        C.addFile("/home/monke/Xrai/maze.g")

        # Add the moving "ego" frame with constraints on its motion
        C.addFrame("ego", "floorwalls") \
            .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
            .setRelativePosition([0.2, 0.2, 0.4]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([0, 1., 1.]) \
            .setContact(1)

        C.addFrame("goal1") \
            .setPosition(C.getFrame('ego').getPosition() + [x_dist, -3, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        C.addFrame("goal2") \
            .setPosition(C.getFrame('goal1').getPosition() + [4, 0, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        q0 = [0.2, 0.2, 0.4]
        qT = C.getFrame('goal1').getPosition()

        # Parameters setup
        ry.params_clear()
        ry.params_add({'rrt/stepsize': 0.05, 'rrt/verbose': 0, 'rrt/maxIters': 15000})

        # PathFinder setup
        rrt = ry.PathFinder()
        rrt.setProblem(C, [q0], [qT])
        # Solve using star_solve
        ret_star_solve = rrt.star_solve()
        if ret_star_solve.feasible:
            star_path_solve = ret_star_solve.x
            total_path_length_star += np.sum([np.linalg.norm(np.array(star_path_solve[j+1]) - np.array(star_path_solve[j])) for j in range(len(star_path_solve)-1)])
            total_evals_star += ret_star_solve.evals
            successful_paths_star += 1

            # Configuration setup
        C = ry.Config()
        C.addFile("/home/monke/Xrai/maze.g")

        # Add the moving "ego" frame with constraints on its motion
        C.addFrame("ego", "floorwalls") \
            .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
            .setRelativePosition([0.2, 0.2, 0.4]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([0, 1., 1.]) \
            .setContact(1)

        C.addFrame("goal1") \
            .setPosition(C.getFrame('ego').getPosition() + [x_dist, -3, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        C.addFrame("goal2") \
            .setPosition(C.getFrame('goal1').getPosition() + [4, 0, 0]) \
            .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
            .setColor([1, 0, 0])

        q0 = [0.2, 0.2, 0.4]
        qT = C.getFrame('goal1').getPosition()

        # Parameters setup
        ry.params_clear()
        ry.params_add({'rrt/stepsize': 0.05, 'rrt/verbose': 0, 'rrt/maxIters': 1000})

        # PathFinder setup
        rrt = ry.PathFinder()
        rrt.setProblem(C, [q0], [qT])
        # Solve using single_star_solve
        ret_single_star = rrt.single_star_solve()
        if ret_single_star.feasible:
            single_star_path = ret_single_star.x
            total_path_length_single_star += np.sum([np.linalg.norm(np.array(single_star_path[j+1]) - np.array(single_star_path[j])) for j in range(len(single_star_path)-1)])
            total_evals_single_star += ret_single_star.evals
            successful_paths_single_star += 1

        # Optionally, you can print progress every 10 iterations
        if i % 10 == 0:
            print(f"Iteration {i+1}: RRT Feasible Paths: {successful_paths_rrt}, PSBI Feasible Paths: {successful_paths_psbi}, Star Feasible Paths: {successful_paths_star}, Single Star Feasible Paths: {successful_paths_single_star}")

    # Calculate averages
    avg_path_length_rrt = total_path_length_rrt / successful_paths_rrt if successful_paths_rrt > 0 else 0
    avg_evals_rrt = total_evals_rrt / successful_paths_rrt if successful_paths_rrt > 0 else 0
    avg_path_length_psbi = total_path_length_psbi / successful_paths_psbi if successful_paths_psbi > 0 else 0
    avg_evals_psbi = total_evals_psbi / successful_paths_psbi if successful_paths_psbi > 0 else 0
    avg_path_length_star = total_path_length_star / successful_paths_star if successful_paths_star > 0 else 0
    avg_evals_star = total_evals_star / successful_paths_star if successful_paths_star > 0 else 0
    avg_path_length_single_star = total_path_length_single_star / successful_paths_single_star if successful_paths_single_star > 0 else 0
    avg_evals_single_star = total_evals_single_star / successful_paths_single_star if successful_paths_single_star > 0 else 0

    # Print results
    print(f"\nAverage Results after 100 iterations at x distance {x_dist }:")
    print(f"RRT - Average Path Length: {avg_path_length_rrt}, Average Evaluations: {avg_evals_rrt}, Successful Paths: {successful_paths_rrt}")
    print(f"PSBI - Average Path Length: {avg_path_length_psbi}, Average Evaluations: {avg_evals_psbi}, Successful Paths: {successful_paths_psbi}")
    print(f"Star - Average Path Length: {avg_path_length_star}, Average Evaluations: {avg_evals_star}, Successful Paths: {successful_paths_star}")
    print(f"Single Star - Average Path Length: {avg_path_length_single_star}, Average Evaluations: {avg_evals_single_star}, Successful Paths: {successful_paths_single_star}")


entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
Iteration 1: RRT Feasible Paths: 1, PSBI Feasible Paths: 1, Star Feasible Paths: 1, Single Star Feasible Paths: 1
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
entered RRT_Star_PathFinder
entered Single_RRT_Star_PathFinder
ente